簡易計算without時間構造化

In [198]:
import pandas as pd 
import os
import numpy as np 
import networkx as nx
from scipy.stats import norm
from scipy.optimize import minimize

In [199]:
# 目的地を16だけにして，リンクデータの構成は全く同じにしてやってみる
df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/easy_test/testnw_link.csv')
df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/easy_test/testnw_node.csv')
df_data = pd.read_csv('/Users/takahiromatsunaga/res2023/easy_test/pydata.csv')

In [200]:
####### リンク接続行列（無向） ####### 
def I_nod(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在リンク（吸収リンクにもつけて良い）
    for i in range(n):
        O = link_data.loc[i, 'o'] # 当該linkのOnode
        D = link_data.loc[i, 'd'] # 当該linkのDnode
        for j in range(n): # リンクiのOorDをOないしDとしているリンクjとは接続
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = I_nod(df_link)

## ?
I[15, :] = 0
I[16, :] = 0

# I[15, 15] = 1  # 時間構造化しない場合は吸収リンクに滞在リンク与えない？？？？？？？

ddata = [16, 17] 
#ddata = [16] 

####### パラメタ初期設定 #######
xdim = 2
x0 = np.zeros(xdim)
x0[0] = -1
x0[-1] = 0.8

In [202]:
print(I)

[[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [208]:
###### 経路選択モデル ####### 
L = len(df_link)
D = len(ddata)

V0 = np.full((L, D), -1) # exp(0)=1よりexp(-1)は小さくなるから?
z0 = np.exp(V0)

###### TS ###### 
# TS = 6

V = V0
z = z0

###### 即時効用行列 ######
def Mset(x): 
    cost = np.exp(df_link['length'] * x[0]) #+ df_link_integrated['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
    cost = pd.concat([cost]*L, axis=1)
    cost = cost.T
    cost_numpy = cost.values # DataFrameをNumPy配列に変換
    return cost_numpy

###### 価値関数 ###### # 時間構造化するなら変わってくるが，，，
def newV(x):
    V = np.zeros((L, D))
    z = np.exp(V)
    beta = x[-1]
    
    for d in range(D): 
        d_linkid = ddata[d]
        d_index = df_link[df_link['linkid'] == d_linkid].index 
        z[d_index, d] = 1                   # 目的地がdの時，dでの期待効用Vd(d)=0から
        M = np.zeros((L, L))
        B = np.zeros((L, 1))
        B[d_index, 0] = 1
        print(B)

        # Mをdごとに更新
        for k in range(L): 
            for a in range(L): 
                Ika = I[k, a] 
                if Ika == 1:                # 接続してなければ無関係（初期値＝0のまま）
                    if a == d_index: 
                        M[k, a] = 1         # 接続先が吸収リンクの場合，その即時効用は 何？exp(0)=1→→負の効用を持たないようにする．他は即時効用は1以下のはず
                    else:
                        M[k, a] = Mset(x)[k, a] # 接続先が普通リンクの場合，その即時効用はMsetそのまま
        
        dL = 100
        zd = z[:, d].reshape(L, 1) 

        # z求解
        count = 0
        while dL >= 0.01:
            zdd = zd.copy()
            # print(zdd)
            zd = M @ (zdd ** beta) + B 
            dL = np.sum(np.abs(zdd - zd)) 
            # dL = np.linalg.norm(zdd - zd, axis=0) 
            count += 1

        # print(M) # ここの出力は問題なさそう

        # z更新
        z[:, d] = zd[:, 0] 
        zd = np.where(zd == 0, 1, zd)  
        one_dim_array = np.log(zd).ravel()   # 1次元配列に
        V[:, d] = one_dim_array 

    return z

In [210]:
x1 = np.array((-7, 0.8)) # ある程度経路効用が負にならないと，接続先の多いリンクが効用高くなる．そいうことがあるのかは知らないけどそうなってる．
print(newV(x1))
# 0.1とかだとおかしくなるわ，最後の目的地に接続するリンクの効用が高くならないわ．

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]]
[[9.11881966e-04 9.11881966e-04 0.00000000e+00 0.00000000e+00
  9.11881966e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [9.11881966e-04 9.11881966e-04 9.11881966e-04 0.00000000e+00
  9.11881966e-04 9.11881966e-04 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 9.11881966e-04 9.11881966e-04 9.11881966e-04
  0.00000000e+00 9.11881966e-04 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.11881966e-04 9.11881966e-04
  0.00000000e+00 0.00000000e+00 0.000

In [211]:
###### 対数尤度関数 ###### 
def loglikelihood(x):
    LL = 0
    beta = x[-1]

    Z = newV(x) 
    # print(Z) 

    # 目的地dごとに計算
    grouped = df_data.groupby('d')
    group = [group for name, group in grouped]
    group = [data.reset_index(drop=True) for data in group]

    for d in range(D):
        d_linkid = ddata[d] 
        d_index = df_link[df_link['linkid'] == d_linkid].index
        d_index = d_index.tolist()
        d_index = d_index[0] 

        # 価値関数
        z = Z[:, d].reshape(L, 1) # 価値関数は固定じゃなくてloglikelihood内で逐次更新→直したらループが終わらないことは無くなった
        z = z ** beta
        ZD = np.tile(z, (1, L))
        ZD = ZD.T

        # Mをdごとに更新（dループの外で問題はなさそう）
        M = np.zeros((L, L))
        for k in range(L): 
            for a in range(L): 
                Ika = I[k, a] 
                if Ika == 1:                            # 接続してなければ無関係（0のまま）
                    if a == d_index: 
                        M[k, a] = 1                     # 吸収リンクの即時効用はexp(0)=1（接続してたら）
                    else:
                        M[k, a] = Mset(x)[k, a]

        # 選択確率
        Mz = (M @ z != 0) * (M @ z) + (M @ z == 0)  
        MZ = np.tile(Mz, (1, L))  
        p = (M * ZD) / MZ
        
        print(p)
        df = group[d]

        for h in range(len(df)):
            pka = 0
            # if a == dd 吸収リンクは遷移のやつに入れてない→が，入れる必要があるのか？？
            k = df.loc[h, 'k']
            a = df.loc[h, 'a']

            if p[k-1, a-1] == 0:
                pka = 1
            else:
                pka = p[k-1, a-1]
            LL += np.log(pka)

    return -LL

In [212]:


###### 推定部分 ###### 
dL = 100
n = 0

x_init = np.zeros(2)
bounds = [(-25, -0.1), (0, 1)] # , (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (0, 1)]
# beta = 1
# 注意．LLを返す関数frに対して数値微分を実行
def fr(x): 
    return -loglikelihood(x)

def hessian(x: np.array) -> np.array:
    h = 10 ** -4 # 数値微分用の微小量
    n = len(x)
    res = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            e_i, e_j = np.zeros(n), np.zeros(n)
            e_i[i] = 1
            e_j[j] = 1
            
            res[i][j] = (fr(x + h * e_i + h * e_j)
            - fr(x + h * e_i - h * e_j)
            - fr(x - h * e_i + h * e_j)
            + fr(x - h * e_i - h * e_j)) / (4 * h * h)
    return res

def tval(x: np.array) -> np.array:
    print(f'hesse行列の逆行列{np.linalg.inv(hessian(x))}')            # inv逆行列を計算！
    print(f'各パラメタの分散{-np.diag(np.linalg.inv(hessian(x)))}')        # 対角成分取り出して-1倍
    return x / np.sqrt(-np.diag(np.linalg.inv(hessian(x))))

#num_random_initializations = 100
#for i in range(num_random_initializations):
    # x0をランダムにする
    #x0 = np.random.uniform(low=-10, high=10, size=(2,))  # Modify the range and size accordingly
    #print(f'{i+1}回目の乱数設定，x0は{x0}です')
dL = 100
    ######### 一旦whileループを外して初期値だけ変えて計算
while dL >= 0.01:
        print(f"<<<<<<<<<<<<<<<<now at {n}th loop in while cond>>>>>>>>>>>>>>>>>>")
        n += 1
        x = x0
            # V = V0 
            # z = z0 # 価値関数固定→結局Vはloglikelihood内で作ることにしたので与える必要がない

                # 構造推定part1：対数尤度関数最大化
        res = minimize(loglikelihood, x, method='L-BFGS-B', bounds = bounds) #, options={"maxiter":10,"return_all":True}) 
        
            # 推定値でパラメタxを更新
        x0 = res.x

        print("x0  =", x0)
        print("lnL =", -1*res.fun)

            # 構造推定part2：価値関数更新
            # z0 = newV(x0) 
            # zz0 = (z0 == 0) * 1 + (z0 != 0) * z0 
            # V0 = np.log(zz0)

                # 収束条件の計算（パラメタが収束しているか）
        dL = np.sum(np.abs(x - x0)) 
        print("dL =", dL)
        # print(n)
    #### 本来ここまでwhileループ

### hesse行列自作バージョン

hhh = hessian(x0)
print(hhh)
print(f'繰り返し回数{n}')
#tval = x0 / np.sqrt(np.diag(hhh))
tval = tval(x0)

### BFGS
"""
hhh = res.hess_inv #.todense()
    # print(n)
tval = x0 / np.sqrt(np.diag(hhh))
"""

L0 = -1 * loglikelihood(x_init) # 初期尤度
LL = -1 * res.fun

#end_time = time.time()
#proc_time = end_time - start_time

    ###### 最終結果の出力 ######
print("計算時間")
#print(proc_time)
print("Inputdata")
#print(search_folder)
print("結果の表示")
print(res)
    #print("説明変数の表示")
    #print(linkv)
    #print("NFXP")
print("初期尤度 = ", L0)
print("最終尤度 = ", LL)
print("ρ値 = ", (L0 - LL) / L0)
print("修正済ρ値 = ", (L0 - (LL - len(x0))) / L0)
print("パラメータ初期値 = ", x_init)
print("パラメータ推定値 = ", x0)
print("時間割引率 = ", x[-1])
print("t値 = ", tval)

# beta=0.6にすると最適値は-8.580になった（下限-25で設定）
# beta=0.8にすると-6.12だ
# beta=0.9にすると-5.006だ
# beta=1にすると初期尤度nanになる-4.18で出た←beta1は信用できない．
# betaも推定させてみる．上限0.9に張り付いた

# beta上限0.99にしてみる．→やはりbeta=0.99に張り付く．
"""
初期尤度 =  -273.47889537719175
最終尤度 =  -88.76375392630297
ρ値 =  0.6754274080130502
修正済ρ値 =  0.6681142294321527
パラメータ初期値 =  [0. 0.]
パラメータ推定値 =  [-4.25710943  0.99      ]
時間割引率 =  0.99
t値 =  [-4.78393893  8.57774557]
"""
# beta上限1にすると崩壊しそうだけど．
"""
初期尤度 =  -273.47889537719175
最終尤度 =  -88.41899201904533
ρ値 =  0.6766880607108833
修正済ρ値 =  0.6693748821299856
パラメータ初期値 =  [0. 0.]
パラメータ推定値 =  [-4.18538293  1.        ]
時間割引率 =  1.0
t値 =  [-4.73587612  8.46022775]
"""
# 1には張り付いたものの尤度はnanになってない．よくわからないすぎる．

# 目的地二つ入れると．beta上限は1．t値出過ぎているけど
"""
)
  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 63
      nit: 0
     njev: 21
   status: 2
  success: False
        x: array([-5.88086299,  1.        ])
初期尤度 =  -309.7573152401913
最終尤度 =  -47.681145408866385
ρ値 =  0.8460693482835311
修正済ρ値 =  0.8396126807518889
パラメータ初期値 =  [0. 0.]
パラメータ推定値 =  [-5.88086299  1.        ]
時間割引率 =  1.0
t値 =  [-169957.20106105   28900.84066295]

"""

<<<<<<<<<<<<<<<<now at 0th loop in while cond>>>>>>>>>>>>>>>>>>
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]]
[[0.36787944 0.36787944 0.         0.         0.36787944 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.36787944 0.36787944 0.36787944 0.         0.36787944 0.36787944
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.36787944 0.36787944 0.36787944 0.         0.36787944
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.36787944 0.36787944 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.36787944 0.36787944 0.         0.         0.36787944 0.
  0.36787944 0.         0.         0.        

[[0.36787944 0.36787944 0.         0.         0.36787944 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.36787944 0.36787944 0.36787944 0.         0.36787944 0.36787944
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.36787944 0.36787944 0.36787944 0.         0.36787944
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.36787944 0.36787944 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.36787944 0.36787944 0.         0.         0.36787944 0.
  0.36787944 0.         0.         0.         0.         0.
  0.         0.36787944 0.         0.         0.        ]
 [0.         0.36787944 0.36787944 0.         0.         0.36787944
  0.36787944 0.36787944 0.

"\n)\n  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'\n     nfev: 63\n      nit: 0\n     njev: 21\n   status: 2\n  success: False\n        x: array([-5.88086299,  1.        ])\n初期尤度 =  -309.7573152401913\n最終尤度 =  -47.681145408866385\nρ値 =  0.8460693482835311\n修正済ρ値 =  0.8396126807518889\nパラメータ初期値 =  [0. 0.]\nパラメータ推定値 =  [-5.88086299  1.        ]\n時間割引率 =  1.0\nt値 =  [-169957.20106105   28900.84066295]\n\n"

In [140]:
x1 = 2 # np.array((2)) #, 0.9))
print(loglikelihood(x1))

d_indexは16のはずInt64Index([15], dtype='int64')
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]


TypeError: 'int' object is not subscriptable

In [ ]:


def loglikelihood_drl(b):
    beta = b[2]
    #beta = 0
    LL = 0
    instant = np.exp(b[0]*dist + b[1]*home) #即時効用マトリックス
    
    for n in range(N): # 個人ごと，timestepごと
        #V=log(vz)
        vz = np.ones((T+1,I))
        
        for t in range(T,0,-1): # 逆向き
            the = instant * (vz[t,:]**beta) #exp(v) * exp(bV)
            zi = the.sum(axis=1)
            vz[t-1,:] = (zi==0)*1 + (zi!=0)*zi
        
        knode = list(route.loc[n*T:(n+1)*T,'k']) # routeは経路情報ということ
        anode = list(route.loc[n*T:(n+1)*T,'a'])
        
        for t in range(T):
            k = int(knode[t])
            a = int(anode[t])
            
            LL += np.log(instant[k,a]) + beta*np.log(vz[t+1,a]) - np.log(vz[t,k])
    return -LL